#  Anomaly Detection in Time-Series

### Use autoencoder to detect anomalies in ECG time-series data.

#### 1. Prepare the data

In [1]:
import pandas as pd

df = pd.read_csv("http://storage.googleapis.com/" 
                 + "download.tensorflow.org/data/ecg.csv",
                 header=None)
print(df.shape)
df.head()

KeyboardInterrupt: 

 The dataset has 140 columns which represents the ECG readings and a labels column which has been encoded to 0 or 1 showing whether the ECG is abnormal or normal.

In [2]:
from src.utils import plotters

plotters.show_traces(df.iloc[0:10, :-1])

Split the data into training and testing sets.

In [3]:
from sklearn.model_selection import train_test_split

# Separate the data and labels
data = df.iloc[:,:-1].values
labels = df.iloc[:,-1].values

# Split the data into training, validation, and test sets
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=21)

# Further split the training data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    train_data, train_labels, test_size=0.25, random_state=21)  # 验证集占训练集的 25%

print("Training set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))

Training set size: 2998
Validation set size: 1000
Test set size: 1000


Investigate the distribution of the data.

In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from plotly import graph_objects as go
from src.utils import plotters

# Standardize and normalize the training data
scaler_standard = StandardScaler()
train_data_standardized = scaler_standard.fit_transform(train_data)

scaler_minmax = MinMaxScaler()
train_data_normalized = scaler_minmax.fit_transform(train_data)

# 比较第一个特征在不同处理方法下的分布
plotters.plot_overlay_density(
    [train_data, train_data_standardized, train_data_normalized],
    ["Original", "Standardized", "Normalized"],
    feature_idx=0,  # 比较第一个特征
    title="Feature 1 Distribution Comparison"
)
# plotters.plot_overlay_density(
#     [train_data, train_data_standardized, train_data_normalized],
#     ["Original", "Standardized", "Normalized"],
#     feature_idx=2,  # 比较第一个特征
#     title="Feature 2 Distribution Comparison"
# )
# plotters.plot_overlay_density(
#     [train_data, train_data_standardized, train_data_normalized],
#     ["Original", "Standardized", "Normalized"],
#     feature_idx=3,  # 比较第一个特征
#     title="Feature 3 Distribution Comparison"
# )

**Why split before normalization/standardization?**

If you use the entire dataset to calculate the mean and standard deviation, information from the test set will "leak" into the training process. This can make the model's performance on the test set seem better than it really is.

In [5]:
# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Create a standard scaler
# scaler = StandardScaler()
scaler = MinMaxScaler(feature_range=(0, 1))  # 将特征缩放到 [0, 1] 范围

# Fit and transform the training data
train_data = scaler.fit_transform(train_data)
val_data = scaler.transform(val_data)
test_data = scaler.transform(test_data)

**TensorFlow 3D Array and Axis Example**

This example demonstrates how the `axis` parameter in TensorFlow's `reduce_sum` function works using a 3D array. The array has a shape of `(2, 3, 4)`, containing 2 matrices, each with dimensions `(3, 4)`. 


**Input Array**

    array_3d = tf.constant([
        [
            [1, 2, 3, 4],    # Row 1 of Matrix 1
            [5, 6, 7, 8],    # Row 2 of Matrix 1
            [9, 10, 11, 12]  # Row 3 of Matrix 1
        ],
        [
            [13, 14, 15, 16],  # Row 1 of Matrix 2
            [17, 18, 19, 20],  # Row 2 of Matrix 2
            [21, 22, 23, 24]   # Row 3 of Matrix 2
        ]
    ], dtype=tf.int32)  # Shape: (2, 3, 4)

**Sum along axis=0:**

        Axis 0: Corresponding elements from the 2 matrices are summed
        resulting in 3 rows and 4 columns.
        [[14, 16, 18, 20],  # Row 1
        [22, 24, 26, 28],  # Row 2
        [30, 32, 34, 36]]  # Row 3

**Sum along axis=1:**

        Axis 1: Each matrix has 3 rows, and the elements of these rows
        are summed, resulting in 4 columns.
        [[15, 18, 21, 24],  # Matrix 1
        [51, 54, 57, 60]]  # Matrix 2

**Sum along axis=2:**

        Axis 2 (dimension=2): Each row has 4 columns, and the elements of these columns
        are summed.
        第 2 维 (axis=2)：每行有 4 列, 4列元素之和。
        [[10, 26, 42],  # Matrix 1
        [58, 74, 90]]  # Matrix 2

Separate the data for normal and abnormal ECGs

In [6]:
# [0 or 1] showing whether the ECG is [abnormal or normal].

#The labels are either 0 or 1, so I will convert them into boolean(true or false) 
train_labels = train_labels.astype(bool)
num_true = np.sum(train_labels)
num_false = len(train_labels) - num_true
print(f"True: {num_true}, False: {num_false}")

val_labels = val_labels.astype(bool)
num_true = np.sum(val_labels)
num_false = len(val_labels) - num_true
print(f"True: {num_true}, False: {num_false}")

test_labels = test_labels.astype(bool)
num_true = np.sum(test_labels)
num_false = len(test_labels) - num_true
print(f"True: {num_true}, False: {num_false}")

# Separate the data for normal ECG from that of abnormal ones
# Normal ECG data
normal_train_data = train_data[train_labels]
normal_val_data = val_data[val_labels]
normal_test_data = test_data[test_labels]

# Abnormal ECG data
abnormal_train_data = train_data[~train_labels]
abnormal_val_data = val_data[~val_labels]
abnormal_test_data = test_data[~test_labels]

# Plot the first 10 normal and abnormal ECGs
plotters.show_traces(pd.DataFrame(normal_train_data).iloc[0:10, :],
                     title="Normal ECGs")
plotters.show_traces(pd.DataFrame(abnormal_train_data).iloc[0:10, :],
                     title="Abnormal ECGs")

True: 1771, False: 1227
True: 588, False: 412
True: 560, False: 440


#### 2. Create autoencoder model

AutoEncoder is an unsupervised Artificial Neural Network that attempts to encode the data by compressing it into the lower dimensions (bottleneck layer or code) and then decoding the data to reconstruct the original input. The bottleneck layer (or code) holds the compressed representation of the input data.

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

# Define the model!
class detector(Model):
  def __init__(self):
    super(detector, self).__init__() 
    # leaky_relu relu PReLU
    self.encoder = tf.keras.Sequential([
                                        layers.Dense(128, activation='relu'),
                                        layers.Dropout(0.2),
                                        layers.Dense(64, activation='relu'),
                                        layers.Dropout(0.2),
                                        layers.Dense(32, activation='relu'),
                                        layers.Dense(16, activation='relu')
    ])
    self.decoder = tf.keras.Sequential([
                                        layers.Dense(32, activation='relu'),
                                        layers.Dropout(0.05),
                                        layers.Dense(64, activation='relu'),
                                        layers.Dense(128, activation='relu'),
                                        layers.Dense(140, activation='tanh')
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

Comparison of Activation Functions

In [8]:
import numpy as np
import plotly.graph_objects as go

# Define activation functions
x = np.linspace(-5, 5, 1000)

activation_functions = {
    'ReLU': np.maximum(0, x),
    'Leaky ReLU': np.where(x > 0, x, 0.01 * x),
    'ELU': np.where(x > 0, x, 1.0 * (np.exp(x) - 1)),
    'SELU': np.where(x > 0, 1.0507 * x, 1.0507 * 1.67326 * (np.exp(x) - 1)),
    'PReLU': np.where(x > 0, x, 0.2 * x),
    'Swish': x / (1 + np.exp(-x)),
    'GELU': 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3))),
    'Softplus': np.log(1 + np.exp(x)),
    'Mish': x * np.tanh(np.log(1 + np.exp(x))),
    'Sigmoid (traditional)': 1 / (1 + np.exp(-x)),
    'Tanh (traditional)': np.tanh(x)
}

# Create the plot
fig = go.Figure()

for name, y in activation_functions.items():
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name=name))

# Customize layout
fig.update_layout(
    title='Comparison of Activation Functions',
    xaxis_title='Input',
    yaxis_title='Output',
    legend_title='Activation Functions',
    template='plotly_white'
)

fig.show()


#### 3. Train the model

In [9]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Assuming `detector` is already defined and normal_train_data, normal_val_data are prepared
autoencoder = detector()

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=10,         # Stop if no improvement for 10 epochs
    restore_best_weights=True
)

# Learning rate scheduler to reduce the learning rate on plateau
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.5,          # Reduce learning rate by a factor of 0.5
    patience=5,          # Wait for 5 epochs with no improvement
    min_lr=1e-6          # Minimum learning rate
)

# Train the model
history = autoencoder.fit(
    normal_train_data,
    normal_train_data,
    epochs=100,
    batch_size=512,
    validation_data=(normal_val_data, normal_val_data),
    callbacks=[early_stopping, lr_scheduler]
)

# Plot training and validation loss using Plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y=history.history['loss'],
        mode='lines',
        name='Training Loss'
    )
)
fig.add_trace(
    go.Scatter(
        y=history.history['val_loss'],
        mode='lines',
        name='Validation Loss'
    )
)
fig.update_layout(
    title='Training and Validation Loss',
    xaxis_title='Epochs',
    yaxis_title='Loss (MSE)',
    template='plotly_white'
)
fig.show()

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.3783 - val_loss: 0.3296 - learning_rate: 0.0010
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.3094 - val_loss: 0.2174 - learning_rate: 0.0010
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1942 - val_loss: 0.1207 - learning_rate: 0.0010
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1108 - val_loss: 0.0790 - learning_rate: 0.0010
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0717 - val_loss: 0.0451 - learning_rate: 0.0010
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0386 - val_loss: 0.0210 - learning_rate: 0.0010
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0191 - val_loss: 0.0169 - learning_rate: 0.0010
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0156 - val_loss: 0.0101 - learning_rate: 0.0010
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0089 - val_loss: 0.0087 - learning_rate: 0.0010
Epoch 10/100
4/4 ━━━━━━━━━━━

Calculate hte threshold with the mean value and standard deviation of the reconstruction loss, or use the 95 percentile.

In [10]:
# Set threshold based on training reconstruction error
train_reconstructed = autoencoder.predict(normal_train_data)
train_reconstruction_error = tf.reduce_mean(
    tf.square(normal_train_data - train_reconstructed),
    axis=1).numpy()
threshold = train_reconstruction_error.mean() + 1 * train_reconstruction_error.std()
print(f"Reconstruction Error Threshold 1 std: {threshold}")

# # Set threshold at the 95th percentile
# threshold = np.percentile(train_reconstruction_error, 95)
# print(f"Reconstruction Error Threshold percentile: {threshold}")


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Reconstruction Error Threshold 1 std: 0.012379196646986862


In [11]:
# Reconstruct the test data to calculate reconstruction error
reconstructed_data = autoencoder.predict(test_data)
reconstruction_error = tf.reduce_mean(
    tf.square(test_data - reconstructed_data),
    axis=1
    ).numpy()

# Separate normal and anomaly samples based on test_labels
normal_indices = (test_labels == 1)
anomaly_indices = (test_labels == 0)

# Plot reconstruction errors and threshold
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.arange(len(reconstruction_error))[normal_indices],
        y=reconstruction_error[normal_indices],
        mode='markers',
        name='Normal Samples',        
        marker=dict(color='blue')
    )
)
fig.add_trace(
    go.Scatter(
        x=np.arange(len(reconstruction_error))[anomaly_indices],
        y=reconstruction_error[anomaly_indices],
        mode='markers',
        name='Anomaly Samples',        
        marker=dict(color='red')
    )
)
fig.add_trace(
    go.Scatter(
        y=[threshold] * len(reconstruction_error), mode='lines', name='Threshold'
    )
)
fig.update_layout(
    title='Reconstruction Errors and Threshold',
    xaxis_title='Sample Index',
    yaxis_title='Reconstruction Error',
    template='plotly_white'
)
fig.show()

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [1]:
from sklearn.metrics import confusion_matrix

# [0 or 1] showing whether the ECG is [abnormal or normal].
# Test dataset: True: 560, False(anomaly): 440

# Use threshold to classify anomalies
y_pred = (reconstruction_error > threshold).astype(int)
y_pred = 1 - y_pred  # 0: Anomaly, 1: Normal
y_true = test_labels

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1]) # 0: Anomaly, 1: Normal
print("Confusion Matrix:\n", cm)

# Extract confusion matrix values
TN, FP = cm[0, 0], cm[1, 0]  # TN: Anomaly correctly classified, FP: Anomaly misclassified as Normal
FN, TP = cm[0, 1], cm[1, 1]  # FN: Normal misclassified as Anomaly, TP: Normal correctly classified

# Correctly reorder the confusion matrix for visualization
# custom_cm = [[TP, FP], [FN, TN]]
custom_cm = [[FN, TN], [TP, FP]]
print("Custom Confusion Matrix:\n", custom_cm)

# Define labels
labels = ["Normal (Positive)", "Anomaly (Negative)"]

fig = go.Figure(
    data=go.Heatmap(
        z=custom_cm,
        x=labels,
        y=labels,
        colorscale="Blues",
        texttemplate="%{z}",
        textfont={"size": 14}
    )
)
fig.update_layout(
    title="Confusion Matrix",
    xaxis_title="Predicted Labels",
    yaxis_title="True Labels",
    template="plotly_white",
    xaxis=dict(tickmode="array", tickvals=[1, 0], ticktext=labels),
    yaxis=dict(tickmode="array", tickvals=[1, 0], ticktext=labels),
)
fig.show()

NameError: name 'reconstruction_error' is not defined

Model evaluation metrics

In [13]:
# Accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Precision
precision = TP / (TP + FP) if (TP + FP) > 0 else 0

# Recall (Sensitivity)
recall = TP / (TP + FN) if (TP + FN) > 0 else 0

# Specificity
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

# F1 Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall (Sensitivity): {recall:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"F1 Score: {f1_score:.2f}")

Accuracy: 0.96
Precision: 0.99
Recall (Sensitivity): 0.94
Specificity: 0.98
F1 Score: 0.96


calculate_probabilities

In [14]:
from sklearn.metrics import precision_recall_curve, auc, roc_curve
import plotly.graph_objects as go

def calculate_probabilities(reconstruction_error, threshold):
    return 1 / (1 + np.exp(-(reconstruction_error - threshold)))

probabilities = calculate_probabilities(reconstruction_error, threshold)

# Min-Max Normalization
min_error = np.min(reconstruction_error)
max_error = np.max(reconstruction_error)
normalized_error = (reconstruction_error - min_error) / (max_error - min_error)

# Z-Score Normalization
mean_error = np.mean(reconstruction_error)
std_error = np.std(reconstruction_error)
z_scores = (reconstruction_error - mean_error) / std_error
standarlized_error = 1 / (1 + np.exp(-z_scores))

# # log error
# log_error = np.log1p(reconstruction_error)  # 使用对数扩展

probabilities = standarlized_error

Plot reconstruction loss distribution

In [15]:
import plotly.graph_objects as go
import numpy as np

y_true_binary = np.array(y_true, dtype=int)

# Negative and Positive samples based on y_true_binary
negative_samples = reconstruction_error[y_true_binary == 0]
positive_samples = reconstruction_error[y_true_binary == 1]

# Create histogram for Negative Samples
hist_negative = go.Histogram(
    x=negative_samples,
    nbinsx=50,
    opacity=0.3,
    name="Negative Samples",
    marker_color="blue"
)

# Create histogram for Positive Samples
hist_positive = go.Histogram(
    x=positive_samples,
    nbinsx=50,
    opacity=0.3,
    name="Positive Samples",
    marker_color="red"
)

# Create figure and layout
fig = go.Figure()
fig.add_trace(hist_negative)
fig.add_trace(hist_positive)

# Update layout
fig.update_layout(
    title="Reconstruction Error Distribution",
    xaxis_title="Reconstruction Error",
    yaxis_title="Frequency",
    barmode="overlay",  # Overlay histograms
    template="plotly_white"
)

# Show figure
fig.show()


The model's ability to distinguish between positive and negative samples is insufficient:

- This might be because the autoencoder's learning capacity is limited and it fails to effectively capture the differences between positive and negative samples.

- Adding more layers or neurons to the encoder part of the autoencoder can enhance the model's ability to represent the data.

Plot precision-recall curve

In [16]:
y_true_binary = np.array(y_true, dtype=int)

# Calculate precision-recall values
precision_values, recall_values, _ = precision_recall_curve(y_true_binary, probabilities)
pr_auc = auc(recall_values, precision_values)

# Calculate ROC curve values
fpr, tpr, _ = roc_curve(y_true_binary, probabilities)
roc_auc = auc(fpr, tpr)

# Plot PR curve
fig_pr = go.Figure()
fig_pr.add_trace(go.Scatter(
    x=recall_values, y=precision_values,
    mode='lines', name='PR Curve',
    line=dict(color='blue', width=2)
))
fig_pr.add_trace(go.Scatter(
    x=[0, 1], y=[1, 0],
    mode='lines', name='Random Classifier',
    line=dict(dash='dash', color='gray')
))
fig_pr.update_layout(
    title=f'Precision-Recall Curve (AUC: {pr_auc:.2f})',
    xaxis_title='Recall',
    yaxis_title='Precision',
    template='plotly_white'
)
fig_pr.show()

# Plot ROC curve
fig_roc = go.Figure()
fig_roc.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines', name='ROC Curve',
    line=dict(color='red', width=2)
))
fig_roc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines', name='Random Classifier',
    line=dict(dash='dash', color='gray')
))
fig_roc.update_layout(
    title=f'Receiver Operating Characteristic (AUC: {roc_auc:.2f})',
    xaxis_title='False Positive Rate (FPR)',
    yaxis_title='True Positive Rate (TPR)',
    template='plotly_white'
)
fig_roc.show()

# Print AUC values
print(f"PR AUC: {pr_auc:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")


PR AUC: 0.36
ROC AUC: 0.03


#### 4. Evaluate the model on the test set

In [21]:
# Evaluate the model on the test set
test_loss = autoencoder.evaluate(normal_test_data, normal_test_data, verbose=1)
print(f"Test Loss (MSE): {test_loss}")

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0052 
Test Loss (MSE): 0.0050622583366930485


In [22]:
import plotly.graph_objects as go
import numpy as np

enc_img = autoencoder.encoder(normal_test_data)
dec_img = autoencoder.decoder(enc_img)

input_data = normal_test_data[0]
reconstructed_data = dec_img[0]

# 计算误差
error = np.abs(input_data - reconstructed_data)
# Plot the error between the input and the reconstructed data
plotters.compare_org_reconstructed(input_data, reconstructed_data)


In [23]:
enc_img = autoencoder.encoder(abnormal_test_data)
dec_img = autoencoder.decoder(enc_img)

input_data = abnormal_test_data[0]
reconstructed_data = dec_img[0]

# 计算误差
error = np.abs(input_data - reconstructed_data)
# Plot the error between the input and the reconstructed data
plotters.compare_org_reconstructed(input_data, reconstructed_data)

## Save as html file

In [ ]:
%%time
## Save as html file
from src.utils import utils

source_notebook = "./notebooks/05_ECG_data_autoencoder.ipynb"
target_folder = "./docs/reports"

converter = utils.get_notebook_converter(
    source_notebook, target_folder, additional_pdf=False
)
converter.convert()